In [2]:
import sys
import torch
sys.path.append('D:\\My Documents\\projects\\drug\\')
custom_vocab_fp='D:\\My Documents\\projects\\drug\\st\smiles_transformer\data\chembl_vocab.csv'
from nanoGPT.model import GPTConfig, GPT
from st.smiles_transformer.utils import split as split_method
from rdkit import Chem

MODEL_DIR = 'train_smiles'
MASK_CLS = 'ilm.mask.custom.MaskSmiles'

In [3]:
# load pretrained model
block_size = 256 # context of up to 256 previous characters
n_layer = 5 # 6
n_head = 8
n_embd = 128 # 256
dropout = 0.2
bias = False
vocab_size=53 
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                bias=bias, vocab_size=vocab_size, dropout=dropout) # start with model_args from command line

gptconf = GPTConfig(**model_args)
model = GPT(gptconf)
#model.load_state_dict(torch.load('D:\\My Documents\\projects\\drug\\ilm\\train_smiles\\model.pt'))

number of parameters: 4.74M


In [29]:
# Prepare tokenizer

import os
import pickle

import ilm.tokenize_util
tokenizer = ilm.tokenize_util.Tokenizer['CUSTOM']
ilm.tokenize_util.set_custom_vocab_fp(custom_vocab_fp)
#tokenizer = ilm.tokenize_util.Tokenizer.GPT2
with open(os.path.join(MODEL_DIR, 'additional_ids_to_tokens.pkl'), 'rb') as f:
    additional_ids_to_tokens = pickle.load(f)
additional_tokens_to_ids = {v:k for k, v in additional_ids_to_tokens.items()}
try:
    ilm.tokenize_util.update_tokenizer(additional_ids_to_tokens, tokenizer)
except ValueError:
    print('Already updated')
print(additional_tokens_to_ids)

State (['<pad>', '<unk>', '<eos>', '<sos>', '<mask>', 'c', 'C', '(', ')', 'O', '1', '2', '=', 'N', '@', '[', ']', 'n', '3', 'H', '4', 'F', '-', 'S', 'Cl', '/', 's', 'o', '5', '+', '.', '#', 'Br', '\\', 'P', '6', 'I', '7', 'Na', '-2', '-3', '8', 'B', '9', 'Si', '%10', 'K', '%11', '-4', 'Se', '<|startofinfill|>', '<|endofinfill|>', '<|infill_element|>', '<|startofinfill|>', '<|endofinfill|>', '<|infill_element|>'], {'<pad>': 0, '<unk>': 1, '<eos>': 2, '<sos>': 3, '<mask>': 4, 'c': 5, 'C': 6, '(': 7, ')': 8, 'O': 9, '1': 10, '2': 11, '=': 12, 'N': 13, '@': 14, '[': 15, ']': 16, 'n': 17, '3': 18, 'H': 19, '4': 20, 'F': 21, '-': 22, 'S': 23, 'Cl': 24, '/': 25, 's': 26, 'o': 27, '5': 28, '+': 29, '.': 30, '#': 31, 'Br': 32, '\\': 33, 'P': 34, '6': 35, 'I': 36, '7': 37, 'Na': 38, '-2': 39, '-3': 40, '8': 41, 'B': 42, '9': 43, 'Si': 44, '%10': 45, 'K': 46, '%11': 47, '-4': 48, 'Se': 49, '<|startofinfill|>': 50, '<|endofinfill|>': 51, '<|infill_element|>': 52}) {50: '<|startofinfill|>', 51: '<|

In [30]:
# Load model



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()
_ = model.to(device)

In [54]:
# Create context

context = "COC1=CC=C(C=C1)N2C(=NN=C2SCCCN3C(=O)C4=CC=CC5=C4C(=CC=C5)C3=O)C6=CC=NC=C6".strip()
ref = context

print("Input", context)
context = split_method(context.strip()).split()
#print(context)
L =len(context)
aList = list(range(L))
import random
from ilm.infer_smiles import infill_with_ilm
new_drug_dict=[]
num_new_samples = 0
ref_sampled = 0
for i in range(100):
    sampled_list = random.sample(aList, 2)
    # Change just one location
    for pos in sampled_list:
        context[pos]='<mask>'

    context_ids = ilm.tokenize_util.encode(context, tokenizer)
    # Replace blanks with appropriate tokens from left to right
    _blank_id = ilm.tokenize_util.encode(['<mask>'], tokenizer)[0]
    #print(_blank_id)
    for pos in sampled_list:
        context_ids[context_ids.index(_blank_id)] = additional_tokens_to_ids['<|infill_element|>']
    output = ilm.tokenize_util.decode(context_ids, tokenizer)
    r = ''.join(output.strip())
    #print("Input to model", r)
    # Temperature > 1 introduces more randomness
    generated = infill_with_ilm(
        model,
        additional_tokens_to_ids,
        context_ids,
        num_infills=32,
        nucleus = 1,
        temperature=1.5)
    for g in generated:
        output = ilm.tokenize_util.decode(g, tokenizer)
        r = ''.join(output.strip())
        #print(i, r)
        m = Chem.MolFromSmiles(r)
        if m is not None:
            print("Before ref", r)
            if r != ref:
               if r not in new_drug_dict:
                   new_drug_dict.append(r)
                   num_new_samples+=1
            else:
               ref_sampled +=1
print(new_drug_dict)
print(ref_sampled)

Input COC1=CC=C(C=C1)N2C(=NN=C2SCCCN3C(=O)C4=CC=CC5=C4C(=CC=C5)C3=O)C6=CC=NC=C6
Before ref COC1=CC=C(C=C1)N2C(=NN=C2SCCCN3C(=O)C4=CC=CC5=C4C(=CC=C5)C3=O)C6=CC=NC=C6
Before ref COC1=CC=C(C=C1)N2C(=NN=C2SCCCN3C(=O)C4=CC=CC5=C4C(=CC=C5)C3=O)C6=CC=NC=C6
Before ref COC1=CC=C(C=C1)N2C(=NN=C2SCCCN3C(=O)C4=CC=CC5=C4C(=CC=C5)C3=O)C6=CC=NC=C6
Before ref COC1=CC=C(C=C1)N2C(=NN=C2SCCCN3C(=O)C4=NC=CC5=C4C(=CC=C5)C3=O)C6=CC=NC=C6
Before ref COC1=CC=C(C=C1)N2C(=NN=C2SCCCN3C(=O)C4=CC=CC5=C4C(=CC=C5)C3=O)C6=CC=NC=C6
Before ref COC1=CC=C(C=C1)N2C(=NN=C2SCCCN3C(=O)C4=CC=CC5=C4C(=CC=C5)C3=O)C6=CC=NC=C6
Before ref COC1=CC=C(C=C1)N2C(=NN=C2SCCCN3C(=O)C4=CC=CC5=C4C(=CC=N5)C3=O)C6=CC=NC=C6
Before ref COC1=CC=C(C=C1)N2C(=NN=C2SCCCN3C(=O)C4=CC=CC5=C4C(=CC=C5)C3=O)C6=CC=NC=C6
Before ref COC1=CC=C(C=C1)N2C(=NN=C2SCCCN3C(=O)C4=CC=CC5=C4C(=CC=C5)C3=O)C6=CC=NC=C6
Before ref COC1=CC=C(C=C1)N2C(=NN=C2SCCCN3C(=O)C4=CC=CC5=C4C(=CC=C5)C3=O)C6=CC=NC=C6
Before ref COC1=CC=C(C=C1)N2C(=NN=C2SCCCN3C(=O)C4=CC=CC5=C4C(=CC=C5)C3

In [42]:
from ilm.infer_smiles import infill_with_ilm

generated = infill_with_ilm(
    model,
    additional_tokens_to_ids,
    context_ids,
    num_infills=64,
    nucleus = 1)
#for g in generated:
#    print('-' * 80)
#    print(ilm.tokenize_util.decode(g, tokenizer))

In [59]:
import torch.nn as nn

emb = nn.Embedding(8,16)
inp = torch.Tensor([[1, 2 ,3], [4,5,6]])
print(inp.size())
out = emb(inp.long())
print(out.size())

torch.Size([2, 3])
torch.Size([2, 3, 16])


In [79]:
import math
from torch.autograd import Variable
from torch.nn import functional as F
class PositionalEncoding(nn.Module):
    "Implement the PE function. No batch support?"
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model) # (T,H)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)
    
class TrfmSeq2seq(nn.Module):
    def __init__(self, in_size, hidden_size, nhead=8, n_layers=4, batch_first=True, dropout=0.1):
        super(TrfmSeq2seq, self).__init__()
        self.in_size = in_size
        self.hidden_size = hidden_size
        self.embed = nn.Embedding(in_size, hidden_size)
        self.pe = PositionalEncoding(hidden_size, dropout)
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_size, nhead=nhead, batch_first=batch_first,dropout=dropout)
        self.trfm = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        

    def forward(self, src):
        # src: (B, T)
        embedded = self.embed(src)  # (B,T, H)
        embedded = self.pe(embedded) # (B,T, H)
        out = self.trfm(embedded) # (B,T, H)
        print(src.size(), out.size())
        return out # (B,T,V)

In [80]:
m = TrfmSeq2seq(8, 16)
inp = torch.Tensor([[1, 2 ,3], [4,5,6]]).long()
m(inp)

torch.Size([2, 3]) torch.Size([2, 3, 16])


tensor([[[-1.6497,  0.1046, -1.1334, -0.8931,  1.6584, -0.3894, -0.7980,
          -0.1967,  1.8961, -0.5315,  1.5660,  0.7552, -0.7275,  0.3149,
          -0.1863,  0.2101],
         [-0.7765,  0.9591, -0.2496,  0.6830, -0.1514, -0.3222,  0.4608,
          -0.6958, -1.1900,  2.8837,  0.7874,  0.1602, -1.3347, -0.1818,
          -0.0690, -0.9632],
         [-0.5403, -0.0924, -1.1236,  1.0544,  1.3352, -0.5160, -0.2795,
          -1.8338,  1.5228, -0.4185,  0.6669,  1.0725, -1.6530,  0.9955,
          -0.1437, -0.0463]],

        [[-0.2631, -1.7040, -0.4537, -0.1999,  1.7409,  1.0370, -0.9161,
          -0.1871, -0.3922, -0.9199,  2.1893,  0.0817, -0.3573, -0.7940,
           1.1147,  0.0236],
         [-1.1230, -0.0374, -0.2310,  0.3391,  2.3228, -1.3117, -0.1088,
          -0.9919,  0.2727,  0.3949,  2.2534,  0.1860, -0.3998, -0.3177,
          -0.4291, -0.8185],
         [-0.4133, -0.6562, -0.0973, -1.0399,  2.5576, -0.4564, -1.3055,
           0.0814,  0.8512, -0.1656,  1.7037,  0.8